In [1]:
import numpy as np
%matplotlib ipympl
from matplotlib import pyplot, colors
import ipywidgets

from pygetm import vertical_coordinates

In [ ]:
D = np.linspace(5.0, 500.0, 50)

fig, (ax_sigma, ax_gvc) = pyplot.subplots(ncols=2, figsize=(10, 5))
for ax in (ax_sigma, ax_gvc):
    ax.set_xlabel("bottom depth (m)")
    ax.set_ylabel("depth (m)")
    ax.set_ylim(D[-1], 0.0)
    ax.set_xlim(0.0, D[-1])
    ax.set_facecolor((0.9, 0.9, 0.9))
artists = []
ax_sigma.set_title("zoomed sigma coordinates")
ax_gvc.set_title("generalized vertical coordinates")

cm = colors.ListedColormap(["white"])


def update(nz=30, ddl=0.5, ddu=0.75, Dgamma=40.0, gamma_surf=True):
    # Remove all exisiting elements from the axes
    while artists:
        artists.pop().remove()

    # Sigma coordinates
    vc_sigma = vertical_coordinates.Sigma(nz, ddl, ddu)
    h = vc_sigma(D[np.newaxis, :])[:, 0, :]
    z = np.zeros((nz + 1, h.shape[1]))
    z[1:] = h.cumsum(axis=0)
    dummy = np.zeros((z.shape[0] - 1, z.shape[1] - 1))
    pc = ax_sigma.pcolormesh(D[:], z[-1] - z, dummy, ec="k", lw=0.1, cmap=cm)
    artists.append(pc)

    # Generalized vertical coordinates
    # This may fail for some or all water depths for certain combinations of ddl and ddu
    error = None
    try:
        vc = vertical_coordinates.GVC(nz, ddl, ddu, Dgamma=Dgamma, gamma_surf=gamma_surf)
    except Exception as e:
        error = str(e)
    if error is None and vc.D_max > D[0]:
        imax = D.searchsorted(vc.D_max)
        h = vc(D[np.newaxis, :imax])[:, 0, :]
        z = np.zeros((nz + 1, h.shape[1]))
        z[1:] = h.cumsum(axis=0)
        dummy = np.zeros((z.shape[0] - 1, z.shape[1] - 1))
        pc = ax_gvc.pcolormesh(D[:imax], z[-1] - z, dummy, ec="k", lw=0.1, cmap=cm)
        artists.append(pc)
        if imax < D.size:
            artists.append(ax_gvc.axvspan(vc.D_max, D[-1], fc="r"))
    else:
        artists.append(ax_gvc.axvspan(0.0, D[-1], fc="r"))
    if error is not None:
        artists.append(
            ax_gvc.text(
                0.5,
                0.5,
                error,
                ha="center",
                va="center",
                transform=ax.transAxes,
                wrap=True,
            )
        )

    fig.canvas.draw()


ipywidgets.interact(
    update, nz=(2, 100), ddl=(0.0, 3.0), ddu=(0.0, 3.0), Dgamma=(0.01, 100.0)
);